In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        (os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Run this cell if packages missing. On Kaggle this is usually allowed.
!pip install --upgrade pip --quiet
!pip install timm==0.9.2 albumentations==1.4.3 scikit-image scikit-learn --quiet


In [ ]:
import os, sys, time, math, random
from glob import glob
from pathlib import Path
from typing import List, Tuple, Optional

import numpy as np
import cv2
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2

from skimage.filters import threshold_otsu
from sklearn.cluster import KMeans

# reproducibility
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
if torch.cuda.is_available(): torch.cuda.manual_seed_all(SEED)


In [ ]:
CONFIG = {
    # Point DATA_DIR to folder that contains all images (can be nested class-wise)
    'DATA_DIR': '/kaggle/input/brain-tumor-mri-dataset/Training',  

    # allowed image extensions
    'IMG_EXTS': ('.png', '.jpg', '.jpeg'),

    # image & training params
    'IMG_SIZE': 256,
    'NUM_CLASSES': 4,      # include background as class 0 if applicable
    'BATCH_SIZE': 8,       # reduce if OOM
    'EPOCHS': 12,
    'LR': 1e-4,
    'DEVICE': 'cuda' if torch.cuda.is_available() else 'cpu',

    # backbone
    'BACKBONE': 'swin_tiny_patch4_window7_224', # smaller default
    'PRETRAINED': True,

    # training utils
    'CHECKPOINT_DIR': '/kaggle/working/checkpoints',
    'GENERATED_MASKS_DIR': '/kaggle/working/masks_auto',
    'MODE': 'auto_mask',   # we will auto-generate masks
    'NUM_WORKERS': 0       # set 0 for Kaggle kernel stability; change if desired
}

os.makedirs(CONFIG['CHECKPOINT_DIR'], exist_ok=True)
os.makedirs(CONFIG['GENERATED_MASKS_DIR'], exist_ok=True)

print("Device:", CONFIG['DEVICE'])
print("Images root:", CONFIG['DATA_DIR'])
print("Masks saved to:", CONFIG['GENERATED_MASKS_DIR'])


In [ ]:
def collect_images(root_dir: str, exts=('.png','.jpg','.jpeg')) -> List[str]:
    p = Path(root_dir)
    files = []
    for ext in exts:
        files += [str(x) for x in p.rglob(f'*{ext}')]
    files = sorted(files)
    return files

def read_image(path: str, size: int) -> np.ndarray:
    img = cv2.imread(path)
    if img is None:
        raise FileNotFoundError(f"Image not found or unreadable: {path}")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if (img.shape[0], img.shape[1]) != (size, size):
        img = cv2.resize(img, (size, size), interpolation=cv2.INTER_LINEAR)
    return img


In [ ]:
def generate_pseudo_mask(img: np.ndarray, num_classes: int, sample_limit=20000) -> np.ndarray:
    """
    img: HxWx3 RGB (resized already)
    returns HxW uint8 mask with values 0..num_classes-1
    """
    h, w, _ = img.shape
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    try:
        t = threshold_otsu(gray)
        bin_mask = (gray > t).astype(np.uint8)
    except Exception:
        _, bin_mask = cv2.threshold(gray, 0, 1, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # morphological cleaning
    kernel = np.ones((5,5), np.uint8)
    clean = cv2.morphologyEx(bin_mask.astype(np.uint8), cv2.MORPH_OPEN, kernel)
    clean = cv2.morphologyEx(clean, cv2.MORPH_CLOSE, kernel)

    if num_classes == 2:
        return clean.astype(np.uint8)

    # Multi-class: KMeans clustering on RGB of whole image (fast sampling)
    Z = img.reshape(-1, 3).astype(np.float32)
    n_samples = min(sample_limit, Z.shape[0])
    # sample indices without replacement
    sample_idx = np.random.choice(Z.shape[0], n_samples, replace=False)
    sample = Z[sample_idx]
    K = num_classes
    km = KMeans(n_clusters=K, n_init=3, random_state=SEED)
    km.fit(sample)
    labels = km.predict(Z).reshape(h, w).astype(np.uint8)

    # choose cluster with lowest mean intensity as background
    centers = km.cluster_centers_
    center_means = centers.mean(axis=1)
    bg_cluster = int(np.argmin(center_means))

    remapped = np.zeros_like(labels)
    cur = 1
    for c in range(K):
        if c == bg_cluster:
            remapped[labels==c] = 0
        else:
            remapped[labels==c] = cur
            cur += 1
            if cur >= num_classes:
                cur = num_classes - 1
    return remapped.astype(np.uint8)

def save_mask(mask: np.ndarray, out_path: str):
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    # write as 8-bit single channel image
    cv2.imwrite(out_path, mask.astype(np.uint8))


In [ ]:
images = collect_images(CONFIG['DATA_DIR'], CONFIG['IMG_EXTS'])
print(f"Found {len(images)} images under {CONFIG['DATA_DIR']}")

if len(images) == 0:
    raise SystemExit(f"No images found in {CONFIG['DATA_DIR']}. Check path.")

# generate masks (skips if mask already exists)
for i, img_path in enumerate(images):
    fname = os.path.basename(img_path)
    out_mask = os.path.join(CONFIG['GENERATED_MASKS_DIR'], fname)
    if os.path.exists(out_mask):
        if i % 200 == 0:
            print(f"[{i}] mask exists, skipping {fname}")
        continue
    try:
        img = read_image(img_path, CONFIG['IMG_SIZE'])
        mask = generate_pseudo_mask(img, CONFIG['NUM_CLASSES'])
        save_mask(mask, out_mask)
    except Exception as e:
        print(f"Failed to generate mask for {img_path} -> {e}")
    if (i+1) % 200 == 0 or i==len(images)-1:
        print(f"[{i+1}/{len(images)}] processed {fname}")

print("Mask generation completed. Masks saved in:", CONFIG['GENERATED_MASKS_DIR'])


In [ ]:
class MedicalSegDataset(Dataset):
    def __init__(self, image_paths: List[str], mask_dir: str, img_size:int=256, transform=None):
        self.image_paths = image_paths
        self.mask_dir = mask_dir
        self.img_size = img_size
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        fname = os.path.basename(img_path)
        mask_path = os.path.join(self.mask_dir, fname)

        img = read_image(img_path, self.img_size)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if mask is None:
            mask = np.zeros((self.img_size, self.img_size), dtype=np.uint8)

        if self.transform:
            aug = self.transform(image=img, mask=mask)
            img = aug['image']
            mask = aug['mask']
        else:
            img = torch.from_numpy(img.transpose(2,0,1)).float() / 255.0
            mask = torch.from_numpy(mask).long()

        return img, mask


In [ ]:
class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
        )
    def forward(self,x): return self.conv(x)

class UpConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.up = nn.Sequential(
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),
            ConvBlock(in_ch, out_ch)
        )
    def forward(self,x): return self.up(x)

class SwinGRUSegmenter(nn.Module):
    def __init__(self, backbone_name='swin_tiny_patch4_window7_224', pretrained=True, num_classes=4):
        super().__init__()
        # create backbone (features only)
        self.backbone = timm.create_model(backbone_name, pretrained=pretrained, features_only=True, out_indices=(0,1,2,3))
        feats = self.backbone.feature_info.channels()
        # channel reducers
        self.reduce4 = nn.Conv2d(feats[-1], 512, 1)
        self.reduce3 = nn.Conv2d(feats[-2], 256, 1)
        self.reduce2 = nn.Conv2d(feats[-3], 128, 1)
        self.reduce1 = nn.Conv2d(feats[-4], 64, 1)

        self.gru_in_dim = feats[-2]
        self.gru = nn.GRU(input_size=self.gru_in_dim, hidden_size=256, batch_first=True)
        self.gru_proj = nn.Linear(256, 256)

        self.dec4 = ConvBlock(512, 256)
        self.up3 = UpConv(256+256, 128)
        self.up2 = UpConv(128+128, 64)
        self.up1 = UpConv(64+64, 32)
        self.final_conv = nn.Sequential(nn.Conv2d(32,32,3,padding=1), nn.ReLU(inplace=True), nn.Conv2d(32, CONFIG['NUM_CLASSES'],1))

    def forward(self,x):
        feats = self.backbone(x)
        s1,s2,s3,s4 = feats
        r4 = self.reduce4(s4); r3 = self.reduce3(s3); r2 = self.reduce2(s2); r1 = self.reduce1(s1)
        d4 = self.dec4(r4)
        b,c,h,w = r3.shape
        tokens = r3.view(b,c,h*w).permute(0,2,1)
        gru_out, _ = self.gru(tokens)
        pooled = gru_out.mean(dim=1)
        pooled = self.gru_proj(pooled).unsqueeze(-1).unsqueeze(-1)
        pooled = pooled.expand(-1,-1,d4.shape[2],d4.shape[3])
        d4 = d4 + pooled
        u3 = F.interpolate(d4, scale_factor=2, mode='bilinear', align_corners=False); u3 = torch.cat([u3, r3], dim=1); u3 = self.up3(u3)
        u2 = F.interpolate(u3, scale_factor=2, mode='bilinear', align_corners=False); u2 = torch.cat([u2, r2], dim=1); u2 = self.up2(u2)
        u1 = F.interpolate(u2, scale_factor=2, mode='bilinear', align_corners=False); u1 = torch.cat([u1, r1], dim=1); u1 = self.up1(u1)
        out = self.final_conv(u1)
        return out


In [ ]:
# transforms
train_tf = A.Compose([
    A.Resize(CONFIG['IMG_SIZE'], CONFIG['IMG_SIZE']),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.Normalize(),
    ToTensorV2(),
])
val_tf = A.Compose([
    A.Resize(CONFIG['IMG_SIZE'], CONFIG['IMG_SIZE']),
    A.Normalize(),
    ToTensorV2(),
])

all_images = collect_images(CONFIG['DATA_DIR'], CONFIG['IMG_EXTS'])
n = len(all_images)
print("Total images:", n)
if n == 0:
    raise SystemExit("No images found. Check CONFIG['DATA_DIR'].")

# split
idxs = np.arange(n)
np.random.shuffle(idxs)
split = int(0.8 * n)
train_imgs = [all_images[i] for i in idxs[:split]]
val_imgs   = [all_images[i] for i in idxs[split:]]

train_ds = MedicalSegDataset(train_imgs, CONFIG['GENERATED_MASKS_DIR'], img_size=CONFIG['IMG_SIZE'], transform=train_tf)
val_ds   = MedicalSegDataset(val_imgs, CONFIG['GENERATED_MASKS_DIR'], img_size=CONFIG['IMG_SIZE'], transform=val_tf)

train_loader = DataLoader(train_ds, batch_size=CONFIG['BATCH_SIZE'], shuffle=True, num_workers=CONFIG['NUM_WORKERS'], pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=CONFIG['BATCH_SIZE'], shuffle=False, num_workers=CONFIG['NUM_WORKERS'], pin_memory=True)

print("Train samples:", len(train_ds), "Val samples:", len(val_ds))


In [ ]:
def dice_loss(pred, target, eps=1e-6):
    pred = F.softmax(pred, dim=1)
    target_onehot = F.one_hot(target, num_classes=pred.shape[1]).permute(0,3,1,2).float()
    inter = (pred * target_onehot).sum(dim=(2,3))
    denom = pred.sum(dim=(2,3)) + target_onehot.sum(dim=(2,3))
    dice = (2*inter + eps)/(denom + eps)
    return 1 - dice.mean()

def iou_score(pred, target, num_classes):
    pred_labels = pred.argmax(dim=1)
    ious = []
    for cls in range(num_classes):
        pred_c = (pred_labels == cls)
        target_c = (target == cls)
        inter = (pred_c & target_c).sum().item()
        union = (pred_c | target_c).sum().item()
        if union == 0:
            ious.append(1.0)
        else:
            ious.append(inter/union)
    return np.mean(ious)

device = CONFIG['DEVICE']
model = SwinGRUSegmenter(backbone_name=CONFIG['BACKBONE'], pretrained=CONFIG['PRETRAINED'], num_classes=CONFIG['NUM_CLASSES']).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=CONFIG['LR'])
scaler = torch.cuda.amp.GradScaler() if device.startswith('cuda') else None


In [ ]:
def train_one_epoch():
    model.train()
    total_loss = 0.0
    for imgs, masks in train_loader:
        imgs = imgs.to(device); masks = masks.to(device)
        optimizer.zero_grad()
        if scaler is not None:
            with torch.cuda.amp.autocast():
                logits = model(imgs)
                ce = F.cross_entropy(logits, masks)
                d = dice_loss(logits, masks)
                loss = ce + d
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            logits = model(imgs)
            ce = F.cross_entropy(logits, masks)
            d = dice_loss(logits, masks)
            loss = ce + d
            loss.backward(); optimizer.step()
        total_loss += loss.item() * imgs.size(0)
    return total_loss / len(train_loader.dataset)

def validate_epoch():
    model.eval()
    total_loss = 0.0; total_iou = 0.0
    with torch.no_grad():
        for imgs, masks in val_loader:
            imgs = imgs.to(device); masks = masks.to(device)
            if scaler is not None:
                with torch.cuda.amp.autocast():
                    logits = model(imgs)
            else:
                logits = model(imgs)
            ce = F.cross_entropy(logits, masks)
            d = dice_loss(logits, masks)
            loss = ce + d
            total_loss += loss.item() * imgs.size(0)
            total_iou += iou_score(logits, masks, CONFIG['NUM_CLASSES']) * imgs.size(0)
    return total_loss / len(val_loader.dataset), total_iou / len(val_loader.dataset)

best_iou = 0.0
for epoch in range(1, CONFIG['EPOCHS']+1):
    t0 = time.time()
    train_loss = train_one_epoch()
    val_loss, val_iou = validate_epoch()
    t1 = time.time()
    print(f"Epoch {epoch}/{CONFIG['EPOCHS']} time {t1-t0:.1f}s train_loss {train_loss:.4f} val_loss {val_loss:.4f} val_iou {val_iou:.4f}")
    ckpt = {'epoch':epoch, 'model_state':model.state_dict(), 'optim_state':optimizer.state_dict(), 'val_iou':val_iou}
    torch.save(ckpt, os.path.join(CONFIG['CHECKPOINT_DIR'], f'epoch_{epoch}.pth'))
    if val_iou > best_iou:
        best_iou = val_iou
        torch.save(ckpt, os.path.join(CONFIG['CHECKPOINT_DIR'], 'best.pth'))

print("Training finished. Best val IoU:", best_iou)


In [ ]:
import matplotlib.pyplot as plt

def predict_mask(model, img_path):
    model.eval()
    img = read_image(img_path, CONFIG['IMG_SIZE'])
    inp = torch.from_numpy(img.transpose(2,0,1)).float().unsqueeze(0)/255.0
    inp = inp.to(device)
    with torch.no_grad():
        if scaler is not None:
            with torch.cuda.amp.autocast():
                logits = model(inp)
        else:
            logits = model(inp)
        pred = logits.argmax(dim=1).squeeze(0).cpu().numpy().astype(np.uint8)
    return img, pred

# show 4 samples from val set (or less if fewer)
n_show = min(4, len(val_imgs))
for i in range(n_show):
    p = val_imgs[i]
    img, pred = predict_mask(model, p)
    gt = cv2.imread(os.path.join(CONFIG['GENERATED_MASKS_DIR'], os.path.basename(p)), cv2.IMREAD_GRAYSCALE)
    plt.figure(figsize=(10,4))
    plt.subplot(1,3,1); plt.imshow(img); plt.title("Image"); plt.axis('off')
    plt.subplot(1,3,2); plt.imshow(pred, cmap='jet'); plt.title("Pred mask"); plt.axis('off')
    plt.subplot(1,3,3); plt.imshow(gt, cmap='gray'); plt.title("Pseudo GT mask"); plt.axis('off')
    plt.show()


In [ ]:
# zip generated masks or checkpoints
import shutil
sh_masks = '/kaggle/working/masks_auto.zip'
sh_ckpt  = '/kaggle/working/checkpoints.zip'
shutil.make_archive('/kaggle/working/masks_auto', 'zip', CONFIG['GENERATED_MASKS_DIR'])
shutil.make_archive('/kaggle/working/checkpoints', 'zip', CONFIG['CHECKPOINT_DIR'])
print("Zipped masks to", sh_masks)
print("Zipped checkpoints to", sh_ckpt)
